# Polarisation des commentaires

## Récupération de la base de données et ajout de colonnes de polarités

In [ ]:
#!pip install textblob
#!pip install textblob-fr

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yseul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#import du dataframe avec la colonne clean

#choisir le bon path
path = "C:/Users/yseul/Documents/ENSAE/2A/Info/projet-python-2a/comments.csv"
data = pd.read_csv(path)

In [3]:
def blober(text):
    '''renvoie un score entre -1 et 1.
    Plus le score est proche de 1, plus le commentaire est positif.
    Plus il est proche de -1, plus le commentaire est négatif.'''
    return TextBlob(text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer()).sentiment[0]

blober = np.vectorize(blober)

def sentences(comment):
    '''renvoie une liste des phrases d'un commentaire'''
    tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
    return tokenizer.tokenize(comment)

def polarisation(comment):
    '''renvoie la moyenne des polarisations des phrases d'un commentaire'''
    s = sentences(comment)
    return np.mean(blober(s))

polarisation = np.vectorize(polarisation)

Création de 2 nouvelles colonnes:

    - Polarity : polarité de chaque commentaire, calculée sur le commentaire entier directement
    
    - sentencesPolarity : polarité de chaque commentaire, calculée en prenant la moyenne des polarités sur chaque phrase que comporte le commentaire.

In [4]:
data["Polarity"] = data.loc[:,['textClean']].apply(blober)
data["sentencesPolarity"] = data.loc[:,['textClean']].apply(polarisation)

## Analyse des polarités

In [5]:
datapol = data.groupby(['videoTitle']).mean().sort_values('Polarity', ascending = False).loc[:, ['Polarity', 'sentencesPolarity']]

In [6]:
datapol

,Polarity,sentencesPolarity
videoTitle,,
Cet oiseau prédira les catastrophes ? (expédition) - DBY #81,0.388586,0.316646
Sommes-nous seuls à savoir que l&#39;on existe ? - DBY #72,0.388143,0.366431
L&#39;irréalité virtuelle - DBY #74,0.368912,0.313011
La méthode pour ressusciter des espèces disparues - DBY#84,0.346526,0.290913
Vivre autonome dans l&#39;espace - DBY #80,0.344769,0.317818
Comment ces champis nous ont façonnés - DBY #76,0.344403,0.299452
Vous ne savez pas à quoi ressemble le monde vivant (mais personne en fait) - DBY #75,0.339257,0.321685
L&#39;internet des animaux du futur - DBY #82,0.324370,0.287537
Comment créer une couleur ? - DBY #69,0.320576,0.290094


A faire: des graphiques en tout genres (diagrammes batons, comparaison avec le nbr de likes,...)